In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout, Embedding
import gensim

In [2]:
reviews = ['nice food',
        'amazing restaurant',
        'too good',
        'just loved it!',
        'will go again',
        'horrible food',
        'never go there',
        'poor service',
        'poor quality',
        'needs improvement']

Y = np.array([1,1,1,1,1,0,0,0,0,0])

In [3]:
vocab_size=50
max_length=3
reviews_one_hot=[one_hot(i,vocab_size) for i in reviews]
reviews_encoder=pad_sequences(reviews_one_hot, maxlen=max_length,padding='post')
reviews_encoder

array([[10, 11,  0],
       [10, 38,  0],
       [49, 19,  0],
       [43, 34, 18],
       [19, 26, 21],
       [12, 11,  0],
       [10, 26, 43],
       [20, 26,  0],
       [20, 15,  0],
       [39, 43,  0]], dtype=int32)

In [5]:
embeded_vector_size=5
model= keras.Sequential([
       keras.layers.Embedding(input_dim=vocab_size, output_dim=embeded_vector_size, input_length=max_length,name='Embedding'),
       keras.layers.Flatten(),
       keras.layers.Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Embedding (Embedding)       (None, 3, 5)              250       
                                                                 
 flatten (Flatten)           (None, 15)                0         
                                                                 
 dense (Dense)               (None, 1)                 16        
                                                                 
Total params: 266 (1.04 KB)
Trainable params: 266 (1.04 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [6]:
model.fit (reviews_encoder,Y, epochs=50)

Epoch 1/50
1/1 [==============================] - 1s 683ms/step - loss: 0.6979 - accuracy: 0.2000
Epoch 2/50
1/1 [==============================] - 0s 6ms/step - loss: 0.6967 - accuracy: 0.3000
Epoch 3/50
1/1 [==============================] - 0s 5ms/step - loss: 0.6955 - accuracy: 0.4000
Epoch 4/50
1/1 [==============================] - 0s 7ms/step - loss: 0.6942 - accuracy: 0.6000
Epoch 5/50
1/1 [==============================] - 0s 7ms/step - loss: 0.6930 - accuracy: 0.7000
Epoch 6/50
1/1 [==============================] - 0s 6ms/step - loss: 0.6918 - accuracy: 0.8000
Epoch 7/50
1/1 [==============================] - 0s 7ms/step - loss: 0.6906 - accuracy: 0.8000
Epoch 8/50
1/1 [==============================] - 0s 5ms/step - loss: 0.6894 - accuracy: 0.8000
Epoch 9/50
1/1 [==============================] - 0s 5ms/step - loss: 0.6881 - accuracy: 0.8000
Epoch 10/50
1/1 [==============================] - 0s 7ms/step - loss: 0.6869 - accuracy: 0.8000
Epoch 11/50
1/1 [====================

In [7]:
y_pred=model.predict(reviews_encoder)
y_pred

1/1 [==============================] - 0s 111ms/step


array([[0.52172554],
       [0.5427356 ],
       [0.5319662 ],
       [0.5559747 ],
       [0.50218016],
       [0.4782232 ],
       [0.51714665],
       [0.44607306],
       [0.44843596],
       [0.45095763]], dtype=float32)

In [8]:
test=['poor','great','best','come again']
test_one_hot=[one_hot(i,vocab_size) for i in test]
test_encoder=pad_sequences(test_one_hot, maxlen=max_length,padding='post')
test_encoder

array([[20,  0,  0],
       [26,  0,  0],
       [22,  0,  0],
       [43, 21,  0]], dtype=int32)

In [9]:
y_pred=model.predict(test_encoder)
y=[(lambda x:1  if x>0.5 else 0)(i) for i in y_pred]
y

1/1 [==============================] - 0s 20ms/step


[0, 1, 1, 0]

In [11]:
word_embedding=model.get_layer('Embedding').get_weights()[0]

In [12]:
word_embedding[10] #word Embedding for nice word

array([-0.04661598,  0.03683767, -0.04788278, -0.04008754, -0.0355345 ],
      dtype=float32)

In [20]:
#!pip3 install gensim
#!pip3 install python-Levenshtenin # I couldnt install it
import gensim

In [ ]:
files=tf.keras.utils.get_file(origin='http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Cell_Phones_and_Accessories_5.json.gz',
                             cache_dir='.', extract=True)

In [7]:
!gunzip datasets/reviews_Cell_Phones_and_Accessories_5.json.gz

In [13]:
df=pd.read_json("datasets/reviews_Cell_Phones_and_Accessories_5.json",lines=True)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A30TL5EWN6DFXT,120401325X,christina,"[0, 0]",They look good and stick good! I just don't li...,4,Looks Good,1400630400,"05 21, 2014"
1,ASY55RVNIL0UD,120401325X,emily l.,"[0, 0]",These stickers work like the review says they ...,5,Really great product.,1389657600,"01 14, 2014"
2,A2TMXE2AFO7ONB,120401325X,Erica,"[0, 0]",These are awesome and make my phone look so st...,5,LOVE LOVE LOVE,1403740800,"06 26, 2014"
3,AWJ0WZQYMYFQ4,120401325X,JM,"[4, 4]",Item arrived in great time and was in perfect ...,4,Cute!,1382313600,"10 21, 2013"
4,ATX7CZYFXI1KW,120401325X,patrice m rogoza,"[2, 3]","awesome! stays on, and looks great. can be use...",5,leopard home button sticker for iphone 4s,1359849600,"02 3, 2013"


In [22]:
review_text= df.reviewText.apply(gensim.utils.simple_preprocess)

In [33]:
len(review_text)

194439

In [31]:
model=gensim.models.Word2Vec(
      window=10,
      min_count=2,
      workers=4,    
)
model.build_vocab(review_text, progress_per=500)

In [34]:
model.train(review_text, total_examples=len(review_text), epochs=model.epochs)

(61507031, 83868975)

In [36]:
model.save("datasets/reviews_Cell_Phones_and_Accessories_5.model")

In [54]:
model.wv.similarity("good","great")

0.7799269

In [52]:
model.wv.most_similar("item")

[('product', 0.8759683966636658),
 ('seller', 0.5587958097457886),
 ('vendor', 0.5300081968307495),
 ('shipment', 0.5022819638252258),
 ('unit', 0.5001143217086792),
 ('merchandise', 0.4922395646572113),
 ('shipper', 0.4711645245552063),
 ('amazons', 0.47019287943840027),
 ('promised', 0.46717384457588196),
 ('timely', 0.46614933013916016)]

In [55]:
sport_review_ulr="http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Sports_and_Outdoors_5.json.gz"
tf.keras.utils.get_file(origin=sport_review_ulr, cache_dir='.')

68275834/68275834 [==============================] - 39s 1us/step


'./datasets/reviews_Sports_and_Outdoors_5.json.gz'

In [58]:
!gunzip datasets/reviews_Sports_and_Outdoors_5.json.gz

In [61]:
df_sport=pd.read_json("datasets/reviews_Sports_and_Outdoors_5.json",lines=True)

In [62]:
review_sport=df_sport.reviewText.apply(gensim.utils.simple_preprocess)

In [63]:
len(review_sport)

296337

In [64]:
model=gensim.models.Word2Vec(
    window=10,
    min_count=2,
    workers=4
)
model.build_vocab(review_sport)

In [65]:
model.corpus_count

296337

In [66]:
model.train(review_sport,total_examples=model.corpus_count,epochs=model.epochs)

(91341802, 121496535)

In [67]:
model.wv.most_similar('bad')

[('terrible', 0.6796128153800964),
 ('horrible', 0.6441899538040161),
 ('shabby', 0.621256947517395),
 ('funny', 0.5673673152923584),
 ('upset', 0.5436868667602539),
 ('darn', 0.5323013663291931),
 ('awful', 0.5174518823623657),
 ('greatest', 0.5146384239196777),
 ('sad', 0.5113632082939148),
 ('good', 0.5085117816925049)]

In [68]:
model.wv.vector_size

100

In [73]:
model.wv.vectors.shape

(53233, 100)

In [76]:
model.wv.vectors[0].shape

(100,)

In [77]:
model.wv.get_index('bad')

274

In [83]:
model.wv.key_to_index

In [79]:
model.save("datasets/reviews_Sports_and_Outdoors_5.model")